## What可视化

2D Grand CAM可视化模块

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
from onekey_algo import get_param_in_cwd
# 目录模式
os.makedirs('features/', exist_ok=True)
mydir = get_param_in_cwd('data25d_pattern')
directory = os.path.expanduser(mydir)
samples = [os.path.join(directory, p) for p in os.listdir(directory) if p.endswith('.npy') or p.endswith('.jpg')]
samples

['F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\bianjiePVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\bijinhuanPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caibinPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caichengshunPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caigengyuanPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caihaimingPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caijinlinPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caiqingxiangPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caiyonglinPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caiyoushunPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caiyunboPVP.nii.npy',
 'F:\\\\wlx_OK_PDAC\\\\OS_prediction-V3\\\\data\\\\crop2.5d\\caizhengqi

In [2]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey, target_layer_mapping
from onekey_algo.utils.MultiProcess import MultiProcess
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.use('Agg')

random.shuffle(samples)
sel_model = get_param_in_cwd('dl25d_sel_model')
model_root = os.path.join(get_param_in_cwd('model25d_root'), sel_model) 
model, transformer, device = init_from_onekey(os.path.join(model_root, 'viz'))
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

if os.path.basename(model_root) + '_2D' in target_layer_mapping:
    target_layer = target_layer_mapping[os.path.basename(model_root) + '_2D'] 
else:
    # 未找到默认的target layer，请自己选择!
    target_layer = 'layer3.0.conv2'
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)
viz_dir = os.path.join(model_root, 'Grad-CAM')
os.makedirs(viz_dir, exist_ok=True)
for sample in samples:
    if os.path.exists(os.path.join(viz_dir, os.path.basename(sample))):
        continue
    img = default_loader(sample)
    num_channels = img.shape[-1]
    sample_ = transformer(img)
    sample_  = sample_.view(1, *sample_.size()).to(device)
    res_cam = gradcam(x=sample_, class_idx=None)
    fig, axes = plt.subplots(1, num_channels + 1, figsize=(4 * (num_channels + 1), 4), facecolor='white')
    for i in range(num_channels):
        axes[i].imshow(img[..., i], cmap='gray')
        axes[i].axis('off')
    imshow = axes[num_channels].imshow(-res_cam[0][0].cpu(),cmap='jet')
    axes[num_channels].axis('off')
    cax = fig.add_axes([0.92, 0.17, 0.02, axes[num_channels].get_position().height]) 
    plt.colorbar(imshow, cax=cax)
    plt.savefig(f'{viz_dir}/{os.path.basename(sample).replace(".npy", ".png")}', bbox_inches = 'tight')
    plt.show()
    plt.close(fig)

[2025-11-19 18:19:00 - <frozen onekey_algo.custom.components.comp2>: 234]	INFO	模型参数：{'pretrained': False, 'model_name': 'resnet152', 'num_classes': 1, 'in_channels': 3}


Feature name:  || Module: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kerne